In [1]:
import altair as alt

import pandas as pd

import itertools

import numpy

import warnings
warnings.filterwarnings('ignore')

from IPython.utils import io

import glob

In [2]:
import os
os.chdir('../../../')

In [3]:
# define samples in each age cohort
ped_sera = [2367, 3944, 2462, 2389, 2323, 2388, 3973, 4299, 4584]
teen_sera = [2350, 2365, 2380, 2382, 3866, 3856, 3857, 3862]
adult_sera = ['33C', '34C', '197C', '199C', '215C', '210C', '74C', '68C', '150C', '18C']
infant = ['2462']
# misc_adults = ['AUSAB-05', 'AUSAB-08', 'AUSAB-16', 'AUSAB-07', 'AUSAB-11', 'AUSAB-13']
ferrets = ['ferret_1', 'ferret_2', 'ferret_3']

# get list of lists for samples divided by age group
serum_lists = [ped_sera, teen_sera, adult_sera, infant, ferrets]
age_cohorts = ['0-5 years', '15-18 years', '40-45 years', 'infant', 'ferret']

# adjust this if we want more stringent filtering
min_times_seen = 5

df_list = []

i = 0 # for looping across age cohort definitions

for list in serum_lists:
    for serum in list:
        # reading in values from averaged libA and libB models
        avg_df = pd.read_csv(f'results/antibody_escape/{serum}_icXX_avg.csv'
                            ).query(f"`times_seen` >= {min_times_seen}")
        
        avg_df = avg_df[['site', 'wildtype', 'mutant', 'log2 fold change IC90 mean']]

        avg_df = avg_df.rename(columns={'log2 fold change IC90 mean': 'ic90_mean'})
        
        serum = str(serum) # ped / teen sera automatically read as ints
        avg_df['serum'] = serum
        avg_df['age_cohort'] = age_cohorts[i]

        # also get summed and mean site scores to check AA-level vs site-level metrics
        avg_df['sitewise_ic90_sum'] = avg_df['ic90_mean'].groupby(avg_df['site']).transform('sum')
        avg_df['sitewise_ic90_mean'] = avg_df['ic90_mean'].groupby(avg_df['site']).transform('mean')

        df_list.append(avg_df)

    i+=1

# concat to final df
escape_df = pd.concat(df_list).reset_index(drop=True)

escape_df.head()

,site,wildtype,mutant,ic90_mean,serum,age_cohort,sitewise_ic90_sum,sitewise_ic90_mean
0,-2,D,Y,-0.1270,2367,0-5 years,-0.1270,-0.127000
1,1,Q,R,-0.1413,2367,0-5 years,-0.1413,-0.141300
2,2,K,N,0.0437,2367,0-5 years,0.0437,0.043700
3,3,I,A,0.0551,2367,0-5 years,-0.7658,-0.042544
4,3,I,D,0.0938,2367,0-5 years,-0.7658,-0.042544


In [5]:
escape_df.to_csv('scratch_notebooks/figure_drafts/umap_analysis/230724_escape_df_full.csv')